In [36]:
from sklearn import datasets
import pandas as pd
import numpy as np
iris = datasets.load_iris()
import random
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from src.models_fn.imputer_models import measure_val_error,XGBImputer,MySimpleImputer
from src.data_fn.data_process import simulate_missing_values,replace_infs
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [3]:
data = pd.read_csv('..\data\processed\iris_original.csv')
curr_df = simulate_missing_values(data,output_name=None,prop=0.3)
inf_df = curr_df.copy()
inf_df['target'] = inf_df['target'].map({0.0:'A',1.0:'B',2.0:'C'})

In [4]:
inf_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,NaN,1.4,NaN,NaN
1,4.9,3.0,NaN,NaN,A
2,4.7,NaN,1.3,0.2,NaN
3,4.6,NaN,NaN,0.2,A
4,5.0,3.6,1.4,0.2,NaN
...,...,...,...,...,...
145,6.7,NaN,NaN,2.3,NaN
146,NaN,NaN,5.0,1.9,NaN
147,6.5,3.0,5.2,2.0,C
148,6.2,3.4,NaN,2.3,NaN


In [5]:
dtypes = ["numeric", "numeric", "numeric", "numeric", "categorical"]
cols = data.columns
dtype_list = dict(zip(cols, dtypes))
# data, dtype_list = format_dtypes(data, dtypes=dtypes, cols=cols)
imp = XGBImputer(dtype_list=dtype_list, random_seed=42, verbose=0, cv=2)
res = imp.impute(curr_df)


In [6]:
res

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.100000,2.530076,1.400000,0.266806,0
1,4.900000,3.000000,1.558515,0.308119,0
2,4.700000,3.329635,1.300000,0.200000,0
3,4.600000,3.377945,1.334072,0.200000,0
4,5.000000,3.600000,1.400000,0.200000,0
...,...,...,...,...,...
145,6.700000,3.323565,5.547719,2.300000,2
146,6.445252,2.700566,5.000000,1.900000,2
147,6.500000,3.000000,5.200000,2.000000,2
148,6.200000,3.400000,5.037148,2.300000,2


In [56]:
# error_ = measure_val_error(curr_df,imp,n_folds=1)
from sklearn.metrics import f1_score
n_folds = 1
# cat_cols = ['target','target2']

curr_df_2 = curr_df.dropna(axis=0, how="any").copy()
# curr_df_2['target2'] = curr_df_2['target']

dtype_list = dict(zip(cols, dtypes))
errors = pd.DataFrame(columns=curr_df_2.columns)

# # n-fold cross validation
imp = XGBImputer(dtype_list=dtype_list)

for i in range(n_folds):
    nans = curr_df_2.mask(np.random.random(curr_df_2.shape) < 0.4)
    res = imp.impute(nans)


    num_errors = pd.DataFrame(((res.drop(columns=cat_cols) - curr_df_2.drop(columns=cat_cols)) ** 2).mean(axis=0) ** (1 / 2)).T
    # cat_errors = 
    errors = pd.concat([errors, num_errors], axis=0)

pd.DataFrame(errors.mean(axis=0)).round(2).to_dict()[0]

KeyError: "['target2'] not found in axis"

In [53]:
# f1_score(curr_df_2['target'],res['target'])
curr_df_2['target'].astype(int).dtypes
# res['target'].dtypes

dtype('float64')

In [27]:
if 'XGB' in str(imp.__class__):
    print(2)

2
